In [25]:
import nibabel as nib
import numpy as np
import random
import time
"""
packages that does conventional model fitting
"""
import amico
"""
packages that generate train/test dataset
"""
from FormatData import generate_data, parser as data_parser
"""
packages that produce the rejection shceme
"""
from filter_qa import parser as filter_parser, load_eddy
"""
package to store the intermediate result
"""
import pickle
"""
package to work out RMSE and SSIM
"""
from utils import calc_RMSE, calc_ssim, load_nii_image, filter_mask

<h1>Test AMICO on full volumes on motion-free dataset</h1>
I would test on s03_still

In [2]:
class Namespace:
    """
    a class generate parser for cmd line args
    """
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

In [3]:
# s03 data repo
s03_path = '/home/vw/Desktop/IndividualProject/MedICSS2021_/Data-NODDI/s03_still_reg/'
s03_NDI_path = '/home/vw/Desktop/IndividualProject/MedICSS2021_/Data-NODDI/s03_still_reg/s03_still_reg_NDI.nii'
s03_ODI_path = '/home/vw/Desktop/IndividualProject/MedICSS2021_/Data-NODDI/s03_still_reg/s03_still_reg_ODI.nii'
s03_FWF_path = '/home/vw/Desktop/IndividualProject/MedICSS2021_/Data-NODDI/s03_still_reg/s03_still_reg_FWF.nii'

In [4]:
filter_mask(s03_path, s03_FWF_path)

original mask has: 91131 of nonzero voxels
filtered mask has: 90060 of nonzero voxels


In [5]:
s03_mask_path = '/home/vw/Desktop/IndividualProject/MedICSS2021_/Data-NODDI/s03_still_reg/filtered_mask.nii'

In [6]:
s03_mask = load_nii_image(s03_mask_path)

In [7]:
# load the truth data for subject 3
s03_NDI_img = nib.load(s03_NDI_path)
s03_ODI_img = nib.load(s03_ODI_path)
s03_FWF_img = nib.load(s03_FWF_path)
s03_NDI_affine = s03_NDI_img.affine
s03_ODI_affine = s03_ODI_img.affine
s03_FWF_affine = s03_FWF_img.affine
s03_NDI_img_data = s03_NDI_img.get_fdata()
s03_ODI_img_data = s03_ODI_img.get_fdata()
s03_FWF_img_data = s03_FWF_img.get_fdata()

----------
Auxilary functions for AMICO

In [8]:
def writefile(path, file, combine, savename):
    with open(path+savename, 'w') as fout:
        read_path = path+file
        read_file = open(read_path, 'r')
        lines = read_file.readlines()
        for line in lines:
            temp = line.split()
            temp = [e for e, b in zip(temp, combine) if b == 1]
            fout.write(' '.join(e for e in temp))
            fout.write('\n')
    fout.close()

def writediffusion(path, file, combine, savename):
    img = nib.load(path+file)
    data = img.get_fdata()
    data = data[..., combine==1]
    img = nib.Nifti1Image(data, np.eye(4))
    nib.save(img, path+savename)

In [9]:
def run_amico(subject_path, diffusion, bvals, bvecs, suffix):
    # run amico
    amico.setup()
    # generate a scheme file from the bvals/bvecs files as follows, using the remained bvals and bvecs. Because data rejection is applied
    amico.util.fsl2scheme(subject_path+bvals, subject_path+bvecs)
    ae = amico.Evaluation("/home/vw/Desktop/IndividualProject/MedICSS2021_/Data-NODDI", subject_path.split('/')[-2], verbose=0)
    # load the data
    ae.load_data(dwi_filename = diffusion, scheme_filename = bvals+'.scheme', mask_filename = "filtered_mask.nii", b0_thr = 0)
    # Set model for NODDI and generate the response functions for all the compartments:
    ae.set_model("NODDI")
    ae.generate_kernels()
    ae.load_kernels()
    # model fit.
    ae.fit()
    # ICVF = NDI
    # ISOVF = FWF
    # OD = ODI
    ae.save_results(path_suffix=suffix)

-------

In [ ]:
# full dataset used
combine = np.ones(96, dtype=np.int8)

In [ ]:
# write the b value, b vector files and diffusion files
writefile(s03_path, 'bvals', combine, savename='full_bvals')
writefile(s03_path, 'bvecs', combine, savename='full_bvecs')
writediffusion(s03_path, 'diffusion.nii', combine, savename='full_diffusion')

In [ ]:
# run amico
run_amico(s03_path, 'full_diffusion.nii', 'full_bvals', 'full_bvecs', 'full_volume')

In [ ]:
# get the saved results
icvf_path = '/home/vw/Desktop/IndividualProject/MedICSS2021_/Data-NODDI/s03_still_reg/AMICO/NODDI_full_volume/FIT_ICVF.nii.gz'
isovf_path = '/home/vw/Desktop/IndividualProject/MedICSS2021_/Data-NODDI/s03_still_reg/AMICO/NODDI_full_volume/FIT_ISOVF.nii.gz'
od_path = '/home/vw/Desktop/IndividualProject/MedICSS2021_/Data-NODDI/s03_still_reg/AMICO/NODDI_full_volume/FIT_OD.nii.gz'

icvf_img = nib.load(icvf_path)
icvf = icvf_img.get_fdata()
isovf_img = nib.load(isovf_path)
isovf = isovf_img.get_fdata()
od_img = nib.load(od_path)
od = od_img.get_fdata()

In [ ]:
# analyze the saved results: rmse and ssim
ndi_rmse = calc_RMSE(icvf, s03_NDI_img_data, s03_mask)
odi_rmse = calc_RMSE(od, s03_ODI_img_data, s03_mask)
fwf_rmse = calc_RMSE(isovf, s03_FWF_img_data, s03_mask)
print(ndi_rmse, odi_rmse, fwf_rmse)
RMSE = (ndi_rmse, odi_rmse, fwf_rmse)

ndi_ssim = calc_ssim(icvf, s03_NDI_img_data)
odi_ssim = calc_ssim(od, s03_ODI_img_data)
fwf_ssim = calc_ssim(isovf, s03_FWF_img_data)
print(ndi_ssim, odi_ssim, fwf_ssim)
SSIM = (ndi_ssim, odi_ssim, fwf_ssim)

In [ ]:
path = '/home/vw/Desktop/IndividualProject/MedICSS2021_/Net/new_exp1_results/'
with open(path+'s03_free_96_rmse.pickle', 'wb') as f:
    pickle.dump(RMSE, f)
with open(path+'s03_free_96_ssim.pickle', 'wb') as f:
    pickle.dump(SSIM, f)

--------------
Test on s03 motion data:

In [10]:
s03_motion_path = '/home/vw/Desktop/IndividualProject/MedICSS2021_/Data-NODDI/s03_motion/'

1) I would like to test AMICO fitting on full volume of s03 motion data

In [ ]:
combine = np.ones(96, dtype=np.int8)
writefile(s03_motion_path, 'bvals', combine, savename='full_bvals')
writefile(s03_motion_path, 'bvecs', combine, savename='full_bvecs')
writediffusion(s03_motion_path, 'diffusion.nii', combine, savename='full_diffusion')

In [ ]:
# run amico
run_amico(s03_motion_path, 'full_diffusion.nii', 'full_bvals', 'full_bvecs', 'full_volume')

In [ ]:
# get the saved results
icvf_path = '/home/vw/Desktop/IndividualProject/MedICSS2021_/Data-NODDI/s03_motion/AMICO/NODDI_full_volume/FIT_ICVF.nii.gz'
isovf_path = '/home/vw/Desktop/IndividualProject/MedICSS2021_/Data-NODDI/s03_motion/AMICO/NODDI_full_volume/FIT_ISOVF.nii.gz'
od_path = '/home/vw/Desktop/IndividualProject/MedICSS2021_/Data-NODDI/s03_motion/AMICO/NODDI_full_volume/FIT_OD.nii.gz'

icvf_img = nib.load(icvf_path)
icvf = icvf_img.get_fdata()
isovf_img = nib.load(isovf_path)
isovf = isovf_img.get_fdata()
od_img = nib.load(od_path)
od = od_img.get_fdata()

In [ ]:
# analyze the saved results: rmse and ssim
ndi_rmse = calc_RMSE(icvf, s03_NDI_img_data, s03_mask)
odi_rmse = calc_RMSE(od, s03_ODI_img_data, s03_mask)
fwf_rmse = calc_RMSE(isovf, s03_FWF_img_data, s03_mask)
print(ndi_rmse, odi_rmse, fwf_rmse)
RMSE = (ndi_rmse, odi_rmse, fwf_rmse)

ndi_ssim = calc_ssim(icvf, s03_NDI_img_data)
odi_ssim = calc_ssim(od, s03_ODI_img_data)
fwf_ssim = calc_ssim(isovf, s03_FWF_img_data)
print(ndi_ssim, odi_ssim, fwf_ssim)
SSIM = (ndi_ssim, odi_ssim, fwf_ssim)

In [ ]:
path = '/home/vw/Desktop/IndividualProject/MedICSS2021_/Net/new_exp1_results/'
with open(path+'s03_motion_96_rmse.pickle', 'wb') as f:
    pickle.dump(RMSE, f)
with open(path+'s03_motion_96_ssim.pickle', 'wb') as f:
    pickle.dump(SSIM, f)

------

In [33]:
# use the stringnet threshold to filter off data
t0, t1, r0, r1, out, num = 3.0, 2.0, 3.0, 2.0, 0.05, 96

filter_cmd = '--path ' + s03_motion_path + ' --t0 ' + str(t0) + ' --t1 ' + str(t1) + ' --r0 ' + str(r0) + ' --r1 ' + str(r1) + ' --out ' + str(out) + ' --num ' + str(num)
filter_args = filter_parser().parse_args(filter_cmd.split())
# this should generate move_t0-{t0}_t1-{t1}_r0-{r0}_r1-{r1}_out-{out}.txt file in the path of subject 3 motion for s03 first
load_eddy(filter_args)

In [34]:
movefile = open('/home/vw/Desktop/IndividualProject/MedICSS2021_/Data-NODDI/s03_motion/move_t0-3.0_t1-2.0_r0-3.0_r1-2.0_out-0.05.txt', 'r')
combine = np.array([int(num) for num in movefile.readline().split(' ')[:-1]])

In [35]:
# get the indices of unselected volumes
indices = np.where(combine==0)[0]
len(set(indices))

67

In [36]:
# create a random sample file that to pick a volume from indices and add it to combine; until all volumes were included
rmse_losses = []
ssim_losses = []
for i in range(0,len(indices),5):
    selected = random.sample(set(indices), i)
    temp_combine = combine.copy()
    # add it to the combine file
    for s in selected:
        temp_combine[s] = 1
    selected_vols = str(96-len(indices)+i)
    suffix = selected_vols+'_volume'
    bvals = selected_vols+'_bvals'
    bvecs = selected_vols+'_bvecs'
    diffusion = selected_vols+'_diffusion'
    # write files
    writefile(s03_motion_path, 'bvals', temp_combine, bvals)
    writefile(s03_motion_path, 'bvecs', temp_combine, bvecs)
    writediffusion(s03_motion_path, 'diffusion.nii', temp_combine, diffusion)
    # run amico
    run_amico(s03_motion_path, diffusion+'.nii', bvals, bvecs, suffix=suffix)
    # analyze
    icvf_path = '/home/vw/Desktop/IndividualProject/MedICSS2021_/Data-NODDI/s03_motion/AMICO/NODDI_'+suffix+'/FIT_ICVF.nii.gz'
    isovf_path = '/home/vw/Desktop/IndividualProject/MedICSS2021_/Data-NODDI/s03_motion/AMICO/NODDI_'+suffix+'/FIT_ISOVF.nii.gz'
    od_path = '/home/vw/Desktop/IndividualProject/MedICSS2021_/Data-NODDI/s03_motion/AMICO/NODDI_'+suffix+'/FIT_OD.nii.gz'

    icvf_img = nib.load(icvf_path)
    icvf = icvf_img.get_fdata()
    isovf_img = nib.load(isovf_path)
    isovf = isovf_img.get_fdata()
    od_img = nib.load(od_path)
    od = od_img.get_fdata()

    
    ndi_rmse = calc_RMSE(icvf, s03_NDI_img_data, s03_mask)
    odi_rmse = calc_RMSE(od, s03_ODI_img_data, s03_mask)
    fwf_rmse = calc_RMSE(isovf, s03_FWF_img_data, s03_mask)
    rsme = (ndi_rmse, odi_rmse, fwf_rmse)

    
    ndi_ssim = calc_ssim(icvf, s03_NDI_img_data)
    odi_ssim = calc_ssim(od, s03_ODI_img_data)
    fwf_ssim = calc_ssim(isovf, s03_FWF_img_data)
    ssim = (ndi_ssim, odi_ssim, fwf_ssim)

    path = '/home/vw/Desktop/IndividualProject/MedICSS2021_/Net/new_exp1_results/'
    with open(path+'s03_motion_'+selected_vols+'_rmse.pickle', 'wb') as f:
        pickle.dump(rsme, f)
    with open(path+'s03_motion_'+selected_vols+'_ssim.pickle', 'wb') as f:
        pickle.dump(ssim, f)


-> Precomputing rotation matrices:


   |██████████████████████████████████████████████████████████| 100.0%

   [ DONE ]
-> Writing scheme file to [ /home/vw/Desktop/IndividualProject/MedICSS2021_/Data-NODDI/s03_motion/29_bvals.scheme ]

-> Loading data:
	* DWI signal
		- dim    = 84 x 84 x 50 x 29
		- pixdim = 1.000 x 1.000 x 1.000
	* Acquisition scheme
		- 29 samples, 3 shells
		- 5 @ b=0 , 5 @ b=3000.0 , 9 @ b=1000.0 , 10 @ b=2000.0 
	* Binary mask
		- dim    = 84 x 84 x 50
		- pixdim = 2.500 x 2.500 x 2.500
		- voxels = 90060
   [ 0.0 seconds ]

-> Preprocessing:
	* Normalizing to b0... 

[ min=0.00,  mean=0.63, max=9155.14 ]
	* Keeping all b0 volume(s)
   [ 0.0 seconds ]

-> Creating LUT for "NODDI" model:
   [ LUT already computed. Use option "regenerate=True" to force regeneration ]

-> Resampling LUT for subject "s03_motion":


   |██████████████████████████████████████████████████████████| 100.0%


...

[ min=0.00,  mean=0.58, max=10986.17 ]
	* Keeping all b0 volume(s)
   [ 0.1 seconds ]

-> Creating LUT for "NODDI" model:
   [ LUT already computed. Use option "regenerate=True" to force regeneration ]

-> Resampling LUT for subject "s03_motion":


   |██████████████████████████████████████████████████████████| 100.0%


   [ 135.8 seconds ]

-> Fitting "NODDI" model to 90060 voxels:


   |██████████████████████████████████████████████████████████| 100.0%


   [ 00h 00m 51s ]

-> Saving output to "AMICO/NODDI_94_volume/*":
	- configuration  [OK]
	- FIT_dir.nii.gz  [OK]
	- FIT_ICVF.nii.gz  [OK]
	- FIT_OD.nii.gz  [OK]
	- FIT_ISOVF.nii.gz  [OK]
   [ DONE ]
pred shape: (90060,)
gt has shape: (90060,)
pred shape: (90060,)
gt has shape: (90060,)
pred shape: (90060,)
gt has shape: (90060,)
